In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from skimage.metrics import structural_similarity
import cv2
from imageio import imread
from random import random, choice, randint, sample
from PIL import Image
from PIL import ImageDraw
import os
import random
import matplotlib.pyplot as plt

class Genetic_Algorithm:
    def __init__(self, imgPath, saveName="temp", top=5, maxgroup=200, features=100, epochs=1000):
        # Initialize the genetic algorithm with given parameters
        self.orignal_img, self.type, self.row, self.col = self.OpenImg(imgPath)
        self.max_group = maxgroup
        self.top = top
        self.saveName = saveName
        self.groups = []
        self.features = features
        self.epochs = epochs
        self.group_dict = dict()

        if not os.path.exists(saveName):
            os.mkdir(saveName)
        print("initalizing...")
        for i in range(self.max_group):
            g = []
            for j in range(self.features):
                tmp = [[choice(np.linspace(0, self.row, features)), choice(np.linspace(0, self.col, features))] for x in range(3)]
                tmp.append("#" + ''.join(choice('0123456789ABCDEF') for x in range(6)))

                g.append(tmp.copy())

            self.groups.append(g.copy())

        self.maxg = self.groups[0]
        print("initializing complete！")

        #imgPath: The file path to the input image that the genetic algorithm will try to recreate using polygons.

        #saveName: The name of the folder where the generated images will be saved. If the folder does not exist, it will be created.

        #top: The number of top groups (out of the max_group) to consider for the next generation. These groups have the highest similarity score compared to the original image.

        #maxgroup: The maximum number of groups (population size) to be considered in each generation. Each group consists of a set of polygons that form an image when drawn together.

        #features: The number of polygons in each group. These polygons will be used to recreate the input image.

        #epochs: The number of iterations the genetic algorithm will run to recreate the input image. The algorithm will terminate if it reaches the maximum number of iterations or if the similarity score meets a certain threshold (0.95 in this code).

    def OpenImg(self, imgPath):
        # Open and read the input image
        img = imread(imgPath)
        # print(type(img))
        row, col = img.shape[0], img.shape[1]
        return img, imgPath.split(".")[-1], row, col

    def to_image(self, g):
        # Convert the set of polygons (group) to an image
        array = np.ndarray((self.orignal_img.shape[0], self.orignal_img.shape[1], self.orignal_img.shape[2]), np.uint8)
        array[:, :, 0] = 255
        array[:, :, 1] = 255
        array[:, :, 2] = 255
        newIm1 = Image.fromarray(array)
        draw = ImageDraw.Draw(newIm1)
        for d in g:
            draw.polygon((d[0][0], d[0][1], d[1][0], d[1][1], d[2][0], d[2][1]), d[3])

        return newIm1

    def getSimilar(self, g) -> float:
        # Calculate the structural similarity index between the original image and the generated image
        # array = np.ndarray((self.orignal_img.shape[0], self.orignal_img.shape[1], self.orignal_img.shape[2]), np.uint8)
        # array[:, :, 0] = 255
        # array[:, :, 1] = 255
        # array[:, :, 2] = 255
        # newIm1 = Image.fromarray(array)
        # draw = ImageDraw.Draw(newIm1)
        # for d in g:
        #     draw.polygon((d[0][0], d[0][1], d[1][0], d[1][1], d[2][0], d[2][1]), d[3])
        newIm1 = self.to_image(g)
        ssim = structural_similarity(np.array(self.orignal_img), np.array(newIm1), multichannel=True)
        return ssim

    def draw_image(self, g, cur):
        # Save the generated image
        image1 = self.to_image(g)
        image1.save(os.path.join(self.saveName, str(cur) + "." + self.type))

    def exchange(self, father, mother)->[]:
        # Perform crossover between two parent groups (father and mother)

        min_locate = min(len(father), len(mother))
        n = randint(0, int(randint(25, 100) / 100 * min_locate))

        selected = sample(range(0, min_locate), n)

        for s in selected:
            father[s], mother[s] = mother[s], father[s]

        locat = randint(0, min_locate)
        fhead = father[:locat].copy()
        mhead = mother[:locat].copy()

        ftail = father[locat:].copy()
        mtail = mother[locat:].copy()

        # print(fhead, ftail, mhead, mtail)
        fhead.extend(mtail)
        father = fhead
        mhead.extend(ftail)
        mother = mhead
        return [father, mother]


    def mutation(self, gen):
        # Mutate the given group by changing some of its polygons

        n = int(randint(1, 100) / 1000 * len(gen))

        selected = sample(range(0, len(gen)), n)

        for s in selected:
            tmp = [[choice(np.linspace(0, self.row, 100)), choice(np.linspace(0, self.col, 100))] for x in
                   range(3)]
            tmp.append("#" + ''.join(choice('0123456789ABCDEF') for x in range(6)))
            gen[s] = tmp

        return gen

    def move(self, gen):
        # Move some polygons within the group

        exchage = int(randint(1, 100) / 1000 * len(gen))
        for e in range(exchage):
            sec1 = randint(0, len(gen) - 1)
            sec2 = randint(0, len(gen) - 1)

            gen[sec1], gen[sec2] = gen[sec2], gen[sec1]

        return gen

    def add(self, gen):
        # Move some polygons within the group

        n = int(randint(1, 100) / 1000 * len(gen))

        for s in range(n):
            tmp = [
                [choice(np.linspace(0, self.row, self.features)),
                 choice(np.linspace(0, self.col, self.features))]
                for x in range(3)]
            tmp.append("#" + ''.join(choice('0123456789ABCDEF') for x in range(6)))
            gen.append(tmp)

        return gen

    def cut(self, gen):
        # Remove some polygons from the group

        n = int(randint(1, 100) / 1000 * len(gen))
        selected = sample(range(0, len(gen)), n)

        g = []
        for gp in range(len(gen)):
            if gp not in selected:
                g.append(gen[gp])

        return g

    def variation(self, gen):
        # Generate variations of the given group

        gen = self.mutation(gen.copy())
        gen1 = self.move(gen.copy())
        gen2 = self.add(gen1.copy())
        gen3 = self.cut(gen2.copy())
        return [gen, gen1, gen2, gen3]

    def breeds(self, father, mother):
        # Breed two parent groups and generate offspring
        # crossover
        new1, new2 = self.exchange(father.copy(), mother.copy())
        new3, new4 = self.exchange(father.copy(), mother.copy())
        new5, new6 = self.exchange(father.copy(), mother.copy())
        new7, new8 = self.exchange(father.copy(), mother.copy())
        new9, new10 = self.exchange(father.copy(), mother.copy())
        new11, new12 = self.exchange(father.copy(), mother.copy())
        new13, new14 = self.exchange(father.copy(), mother.copy())
        new15, new16 = self.exchange(father.copy(), mother.copy())

        # mutation
        new17, new18, new19, new20 = self.variation(father.copy())
        new21, new22, new23, new24 = self.variation(mother.copy())

        #cross+mut
        new25, new26, new27, new28 = self.variation(new1.copy())
        new29, new30, new31, new32 = self.variation(new8.copy())

        return [new1, new2, new3, new4, new5, new6, new7, new8, new9, new10, new11, new12, new13, new14, new15, new16, new17, new18, new19, new20, new21, new22, new23, new24, new25, new26, new27, new28, new29, new30, new31, new32]

    def mps_selection(self, groups):
        # Eliminate less fit groups and keep the fittest ones
        self.group_dict.clear()
        for gp in range(len(groups)):
            self.group_dict[gp] = self.getSimilar(groups[gp])
        fitness_values = list(self.group_dict.values())
        fitness_sum = sum(fitness_values)
        relative_fitness = [fit / fitness_sum for fit in fitness_values]
        cumulative_fitness = [sum(relative_fitness[:i + 1]) for i in range(len(fitness_values))]

        selected_indices = []
        current_member = i = 0
        mating_pool_size = self.max_group/4
        r = random.uniform(0, 1 / mating_pool_size)

        while current_member < mating_pool_size:
            while r <= cumulative_fitness[i]:
                selected_indices.append(i)
                r += 1 / mating_pool_size
                current_member += 1
            i += 1

        groups = [groups[idx] for idx in selected_indices]
        fitness_values = [fitness_values[idx] for idx in selected_indices]
        return groups, max(fitness_values)

    def tournament_selection(self, groups):
        # Eliminate less fit groups and keep the fittest ones
        self.group_dict.clear()
        for gp in range(len(groups)):
            self.group_dict[gp] = self.getSimilar(groups[gp])
        
        selected_indices = []
        mating_pool_size = self.max_group/4
        tournament_size = 2
        
        while len(selected_indices) < mating_pool_size:
            tournament = random.sample(range(len(groups)), tournament_size)
            tournament_fitness = [self.group_dict[idx] for idx in tournament]
            best_individual = tournament[tournament_fitness.index(max(tournament_fitness))]
            selected_indices.append(best_individual)
            
        groups = [groups[idx] for idx in selected_indices]
        fitness_values = [self.group_dict[idx] for idx in selected_indices]
        
        return groups, max(fitness_values)

    def uniform_random_selection(self, groups):
        # Eliminate less fit groups and keep the fittest ones
        self.group_dict.clear()
        for gp in range(len(groups)):
            self.group_dict[gp] = self.getSimilar(groups[gp])
        
        selected_indices = []
        current_member = 0
        mating_pool_size = self.max_group/4

        while current_member < mating_pool_size:
            selected_indices.append(random.randint(0, len(groups)-1))
            current_member += 1

        groups = [groups[idx] for idx in selected_indices]
        fitness_values = [self.group_dict[idx] for idx in selected_indices]
        return groups, max(fitness_values)

    def elitism(self, groups):
        # Eliminate less fit groups and keep the fittest ones
        self.group_dict.clear()
        # print(0, self.getSimilar(groups[0]))
        for gp in range(len(groups)):
            self.group_dict[gp] = self.getSimilar(groups[gp])
        # print(1, self.getSimilar(groups[0]))
        # print(1, groups[0])
        self.group_dict = {key: value for key, value in
                           sorted(self.group_dict.items(), key=lambda item: item[1], reverse=True)}

        quarter_length = len(self.group_dict) // 4
        top_quarter_keys = list(self.group_dict.keys())[:quarter_length]

        g = []
        for key in top_quarter_keys:
            g.append(self.groups[key].copy())

        groups = g.copy()
        return groups, list(self.group_dict.values())[0]

    def eliminated(self, groups):
        # Eliminate less fit groups and keep the fittest ones
        self.group_dict.clear()
        # print(0, self.getSimilar(groups[0]))
        for gp in range(len(groups)):
            self.group_dict[gp] = self.getSimilar(groups[gp])
        # print(1, self.getSimilar(groups[0]))
        # print(1, groups[0])
        self.group_dict = {key: value for key, value in
                           sorted(self.group_dict.items(), key=lambda item: item[1], reverse=True)}

        g = []
        for key in list(self.group_dict.keys())[:self.max_group]:
            g.append(self.groups[key].copy())

        groups = g.copy()
        return groups, list(self.group_dict.values())[0]

    def fit(self):
        # Run the genetic algorithm to recreate the input image
        acc_lst = []
        
        self.tournament_selection(self.groups)
        for cur in range(self.epochs):
            breed_n = randint(self.max_group // 2, self.max_group)
            g_f = np.abs(np.array(list(self.group_dict.values())))
            p = g_f / np.sum(g_f)
            for i in range(breed_n):
                f = np.random.choice(list(self.group_dict.keys()), p=p.ravel())
                m = np.random.choice(list(self.group_dict.keys()), p=p.ravel())
                if f < self.max_group and m < self.max_group:
                    self.groups.extend(self.breeds(self.groups[int(f)].copy(), self.groups[int(m)].copy()))

            self.groups, acc = self.eliminated(self.groups.copy())
            print("Epochs :", cur+1, " Acc:", acc)

            acc_lst.append(acc)

            last = self.groups[0]

            if cur % 100 == 0:
                self.draw_image(self.groups[0], cur)

            if acc >= 0.95:
                break
        self.draw_image(self.groups[0], "End")
        plt.plot(range(1, len(acc_lst) + 1), acc_lst)
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.title("Accuracy over Epochs")
        plt.show()

# Define input parameters
path = '/content/drive/MyDrive/Cisc455 Final Project/test.png'
savename = 'test result'

# Create a Genetic_Algorithm instance and run the algorithm
groups = 500
GA = Genetic_Algorithm(path, savename, 5, groups, 50, 500)
GA.fit()
